In [309]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import word_tokenize
import string
from math import ceil
from math import floor
import json
import pandas as pd
from collections import Counter
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

Первый способ - вынуть слова из центра текста (больше по приколу, ну а вдруг повезёт)

In [310]:
extended_punctuation = string.punctuation + '—»«...'
file_texts = []
for some_file in os.listdir('.'): 
    if not some_file.endswith('.txt'):
        continue
    with open(some_file, 'r', encoding='utf-8') as open_file:
        file_texts.append(open_file.read())

In [313]:
def keywords_in_the_middle(some_text, num_words):
    """
    На вход -- строка с текстом some_text, число слов в середине текста
    """
    tokenized_text = [morph.parse(word)[0].normal_form for word in word_tokenize(some_text.lower()) if word not in extended_punctuation]
    if len(tokenized_text) > num_words:
        if num_words > 0:
            return tokenized_text[floor(len(tokenized_text) / 2) - floor(num_words / 2) : ceil(len(tokenized_text) / 2) + floor(num_words / 2)]
        else: 
            return None
    else:
        return tokenized_text

In [314]:
for text in file_texts:
    print(keywords_in_the_middle(text, 10))

['в', 'прошлое', 'год', 'здесь', 'сойти', 'поезд', 'с', 'рельс', 'говорить', 'следователь']
['батюшка', 'пётр', 'андрей', 'шептать', 'савельй', 'стоя', 'за', 'я', 'и', 'толкать', 'я']
['от', 'ты', 'волк', 'не', 'хитро', 'уйти', 'сказка', 'колобок', 'картинка', '3', 'и']
['использовать', 'тест-система', 'государственный', 'научный', 'центр', 'вектор', 'тоже', 'входящий', 'в', 'структура']
['к', 'он', 'прибавиться', 'спорт', 'салон', 'красота', 'развлечение', 'говориться', 'в', 'исследование']


Берем самые частотные леммы с помощью pymorphy2

In [316]:
def most_frequent_lemmas_with_stopwords(some_text, num_most_freq, some_stoplist):
    tags = ['PREP', 'CONJ', 'PRCL', 'INTJ', 'NPRO', 'VERB']
    tokenized_text = [morph.parse(word)[0].normal_form for word in word_tokenize(some_text.lower()) if word.strip() 
                      not in extended_punctuation and morph.parse(word)[0].normal_form not in stoplist and morph.parse(word)[0].tag.POS not in tags]
    return [word_freq_pair[0] for word_freq_pair in Counter(tokenized_text).most_common(num_most_freq)]

In [317]:
stoplist = list(stopwords.words('russian'))
stoplist.extend(['это', 'ваш', 'нешто', 'тебе', 'ежели', 'из-за', 'ту', 'эта', 'дальше', 'свой', 'весь', 'наш'])
for text in file_texts:
    print(most_frequent_lemmas_with_stopwords(text, 10, stoplist))

['гайка', 'денис', 'грузило', 'следователь', 'рельс', 'благородие', 'поезд', 'глаз', 'григорьев', 'господин']
['иван', 'пугачев', 'который', 'савельй', 'марья', 'крепость', 'швабрин', 'бог', 'комендант', 'человек']
['колобок', 'короб', 'сусек', 'заяц', 'песенка', 'волк', 'лис', 'старуха', 'сметана', 'масло']
['тест', 'тестирование', 'лаборатория', 'роспотребнадзор', 'коронавирус', 'биоматериал', 'регион', 'хеликс', 'государственный', 'представитель']
['неделя', 'расход', 'россиянин', 'категория', 'товар', 'салон', 'красота', 'сбербанк', 'трата', 'сравнение']


TF-IDF для униграмм и биграмм

In [318]:
make_tf_idf = TfidfVectorizer(stop_words=stoplist, ngram_range=(1, 2)) 
texts_as_tfidf_vectors = make_tf_idf.fit_transform(file_texts)

In [342]:
def preprocess_for_tfidif(some_text):
    tags = ['PREP', 'CONJ', 'PRCL', 'INTJ', 'NPRO', 'VERB']
    lemmatized_text = [morph.parse(word)[0].normal_form for word in word_tokenize(some_text.lower()) if word.strip() 
                      not in extended_punctuation and morph.parse(word)[0].normal_form not in stoplist and morph.parse(word)[0].tag.POS not in tags]
    return ' '.join(lemmatized_text)

In [343]:
def produce_tf_idf_keywords(some_texts, number_of_words):
    result = []
    texts_as_tfidf_vectors = make_tf_idf.fit_transform(preprocess_for_tfidif(text) for text in some_texts)
    id2word = {i : word for i, word in enumerate(make_tf_idf.get_feature_names())} 
    for text_row in range(texts_as_tfidf_vectors.shape[0]):
        row_data = texts_as_tfidf_vectors.getrow(text_row) 
        words_for_this_text = row_data.toarray().argsort()
        top_words_for_this_text = words_for_this_text[0, : -1*(number_of_words+1) : -1]
        result.append([id2word[w] for w in top_words_for_this_text])
    return result

In [344]:
print(produce_tf_idf_keywords(file_texts, 10))

[['гайка', 'денис', 'грузило', 'следователь', 'рельс', 'поезд', 'благородие', 'григорьев', 'глаз', 'господь'], ['иван', 'пугачев', 'марья', 'савельй', 'марья иван', 'крепость', 'который', 'швабрин', 'бог', 'комендант'], ['колобок', 'колобок колобок', 'сусек', 'лис', 'короб', 'заяц', 'сметана', 'колобок картинка', 'сказка колобок', 'волк'], ['тест', 'тестирование', 'роспотребнадзор', 'лаборатория', 'коронавирус', 'биоматериал', 'регион', 'хеликс', 'представитель', 'государственный'], ['неделя', 'расход', 'россиянин', 'товар', 'категория', 'красота', 'трата', 'салон красота', 'салон', 'сбербанк']]


In [345]:
with open("data/russia_today_1.jsonlines", "r", encoding='utf-8') as read_file:
    rt_1_data = [json.loads(line) for line in read_file]

In [355]:
for item in rt_1_data[:10]:
    print('Эталонные ключевые слова: ', item['keywords'])
    print('Слова из середины текста:', keywords_in_the_middle(item['content'], 10), '\n')

Эталонные ключевые слова:  ['в россии', 'краснодар', 'рфпл', 'ростов', 'фк спартак', 'спорт', 'пфк цска', 'футбол']
Слова из середины текста: ['присмотреться', 'к', 'возможность', 'сельт', 'и', 'тот', 'и', 'другой', 'не', 'привыкнуть', 'играть'] 

Эталонные ключевые слова:  ['в мире', 'евгений малкин', 'канада', 'кубок стэнли', 'нхл', 'россия', 'сша', 'спорт', 'спортсмен', 'хоккей', 'хоккей']
Слова из середины текста: ['играть', 'в', 'меньшинство', 'но', 'большинство', 'питтсбург', 'из', 'преимущество', 'превратиться', 'в'] 

Эталонные ключевые слова:  ['анатолий антонов', 'вашингтон', 'госдеп сша', 'дипломаты', 'иноагент', 'мид', 'мария захарова', 'миротворец', 'посольство', 'сми', 'сша', 'хезер науэрт', 'цензура', 'внешняя политика']
Слова из середины текста: ['rt', 'america', 'зарегистрироваться', 'в', 'качество', 'иностранный', 'агент', 'тогда', 'заместитель', 'руководитель'] 

Эталонные ключевые слова:  ['биржа', 'доллар', 'рубль', 'цифровая экономика', 'биткоин', 'криптовалюта', 

In [428]:
manual_keywords = [] 
full_texts = []

In [429]:
for item in rt_1_data:
    if item['content']:
        manual_keywords.append(item['keywords'])
        full_texts.append(item['content'])

In [430]:
for manual_keywords_group, predicted_keywords_group in zip(manual_keywords[:20], produce_tf_idf_keywords(full_texts[:20], 10)):
    print('Эталонные ключевые слова: ', manual_keywords_group)
    print('Слова из середины текста:', predicted_keywords_group, '\n')

Эталонные ключевые слова:  ['в россии', 'краснодар', 'рфпл', 'ростов', 'фк спартак', 'спорт', 'пфк цска', 'футбол']
Слова из середины текста: ['команда', 'клуб', 'лига', 'болельщик', 'сезон', 'спартак', 'томь', 'игрок', 'цска', 'лига европа'] 

Эталонные ключевые слова:  ['в мире', 'евгений малкин', 'канада', 'кубок стэнли', 'нхл', 'россия', 'сша', 'спорт', 'спортсмен', 'хоккей', 'хоккей']
Слова из середины текста: ['нэшвилл', 'питтсбург', 'матч', 'кубок стэнли', 'стэнли', 'команда', 'серия', 'защитник', 'гость', 'кубок'] 

Эталонные ключевые слова:  ['анатолий антонов', 'вашингтон', 'госдеп сша', 'дипломаты', 'иноагент', 'мид', 'мария захарова', 'миротворец', 'посольство', 'сми', 'сша', 'хезер науэрт', 'цензура', 'внешняя политика']
Слова из середины текста: ['сми', 'мера', 'сша', 'иностранный агент', 'российский', 'агент', 'иностранный', 'посольство', 'российский сми', 'американский'] 

Эталонные ключевые слова:  ['биржа', 'доллар', 'рубль', 'цифровая экономика', 'биткоин', 'криптова

In [431]:
predicted_keywords_tf_idf = produce_tf_idf_keywords(full_texts, 10)

In [432]:
predicted_keywords_in_the_middle = [keywords_in_the_middle(text, 10) for text in full_texts]

In [433]:
predicted_most_frequent_lemmas_with_stopwords = [most_frequent_lemmas_with_stopwords(text, 10, stoplist) for text in full_texts]

In [434]:
def precision_recall_fmeasure_jaccard(manual, predicted):
    """
    считает точность, полноту, F-меру и коэффицент Жаккара
    """
    precisions = []
    recalls = []
    intersections = []
    for index, words_manual in enumerate(manual):
        words_predicted = predicted[index]
        intersection = len(set(words_manual) & set(words_predicted))
        recalls.append(intersection / len(words_manual)) 
        if len(words_predicted) == 0:
            print(len(full_texts[index]))
        precisions.append(intersection / len(words_predicted))
        intersections.append(intersection)
    mean_precision = sum(precisions) / len(precisions)
    mean_recall = sum(recalls) / len(recalls)
    fmeasure = ((2 * mean_recall * mean_precision) / (mean_recall + mean_precision))
    jaccard_index = sum(intersections) / (len([words_manual for words_manual in manual]) + len([words_predicted for words_predicted in predicted]) - sum(intersections))
    return f'Точность: {mean_precision}, полнота: {mean_recall}, F-мера {fmeasure}, коэффицент Жаккара {jaccard_index}'

In [435]:
precision_recall_fmeasure_jaccard(manual_keywords, predicted_keywords_tf_idf)

'Точность: 0.11011011011010972, полнота: 0.11226604304272218, F-мера 0.1111776257102956, коэффицент Жаккара 1.2249443207126949'

In [436]:
precision_recall_fmeasure_jaccard(manual_keywords, predicted_keywords_in_the_middle)

'Точность: 0.02687882687882691, полнота: 0.030870581882303676, F-мера 0.028736745323064892, коэффицент Жаккара 0.16365754222481071'

In [437]:
precision_recall_fmeasure_jaccard(manual_keywords, predicted_most_frequent_lemmas_with_stopwords)

'Точность: 0.14583750417083668, полнота: 0.14840256643236568, F-мера 0.14710885472991353, коэффицент Жаккара 2.6863468634686347'